## RRDV con Gensim

Importamos librerías de utilidad.

In [1]:
import pandas as pd
from pathlib import Path
from typing import List

from gensim.models import TfidfModel
from gensim import corpora, similarities

from documents import load_docs, Document

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Cargamos nuestros documentos.

In [2]:
all_docs = load_docs(Path("./data/docs-raw-texts"))

Vamos a sintetizar todo en una clase llamada `GensimSearch`. Su similitud con la que construimos manualmente para RRDV es evidente, solo que el cálculo del TF-IDF y las similitudes son mucho más directas.

In [10]:
class GensimSearch:
    def __init__(self, docs: List[Document]):
        """
        Inicializa el RRDV con funciones de Gensim con una lista de documentos.

        Args:
            docs (List[Document]): Lista de objetos Document para construir el índice TF-IDF.
        """
        self.docs = docs

        # Implemente el concepto de un Dictionary, es decir, mapear palabras a enteros.
        self.dictionary = corpora.Dictionary(all_docs)

        # El corpus se construirá con el método doc2bow de gensim, que cuenta las ocurrencias de cada término en cada documento.
        self.corpus = [self.dictionary.doc2bow(doc) for doc in all_docs]

        # Construimos el modelo TF-IDF de Gensim con nuestro corpus.
        self.model = TfidfModel(self.corpus)

        # Calcula la similitud coseno para nuestro corpus y el modelo TF-IDF.
        self.index = similarities.MatrixSimilarity(self.model[self.corpus])
        
    def search(self, query_document: Document, min_similarity: float = 0.0) -> pd.DataFrame:
        """
        Realiza una búsqueda para encontrar documentos similares al documento de consulta.

        Args:
            query_document (Document): Documento de consulta para buscar similitudes.
            min_similarity (float): Umbral mínimo de similitud para filtrar resultados. Por defecto es 0.

        Returns:
            pd.DataFrame: DataFrame con documentos relevantes y sus similitudes.
        """
        # Tenemos que aplicar el mismo procesamiento a la consulta, es decir, conteo de términos (doc2bow), aplicarle el modelo TF-IDF, y calcular similitud coseno con el índice ya construido.
        query_bow = self.dictionary.doc2bow(query_document)
        query_tfidf = self.model[query_bow]
        sims = self.index[query_tfidf]

        # Crear un DataFrame con los resultados de similitud
        results = pd.DataFrame({
            'similarity': sims,
            'doc': self.docs
        }, index=[doc.name for doc in self.docs])
        
        # Ordenar los resultados por similitud de mayor a menor
        results.sort_values(by='similarity', ascending=False, inplace=True)

        # Filtrar los resultados por similitud mínima
        results = results[results['similarity'] > min_similarity]
        
        return results
    
    def evaluate_search(self, queries: List[Document], output_path: Path):
        """
        Evalúa las consultas y escribe los resultados en un archivo de salida.
    
        Args:
            queries (List[Document]): Lista de documentos de consulta para evaluar.
            output_path (Path): Ruta del archivo donde se guardarán los resultados.
        """
        with open(output_path, 'w') as output_file:
            for query in queries:
                relevant_docs = self.search(query_document=query)    
                result_texts = [f'{doc_name}:{row.similarity}' for doc_name, row in relevant_docs.iterrows()]
                output_file.write(f"{query.name}\t{','.join(result_texts)}\n")

Construimos nuestra clase con nuestros documentos.

In [11]:
gensim_search = GensimSearch(all_docs)

Y calculamos todos los queries, almacenándolos en el formato especificado.

In [12]:
all_queries = load_docs(Path("./data/queries-raw-texts"))
gensim_search.evaluate_search(all_queries, output_path='./data/GESIM-consultas_resultados')